### BeautifulSoup 사용해서 저스트와치 상세페이지 포스터 이미지 다운로드

In [2]:
# csv파일 읽어오기 - kino_netflix
import pandas as pd
kino_netflix_all = pd.read_csv("kino/kino_netflix_all.csv").drop(["Unnamed: 0"], axis=1)
kino_netflix_all = kino_netflix_all["netflix_key-data"] 
kino_netflix_all

0       70457
1        3879
2       96657
3       65280
4        8065
        ...  
3227    64567
3228    97147
3229    72005
3230    72670
3231    77006
Name: netflix_key-data, Length: 3232, dtype: int64

In [5]:
title_list =[]
openingdate_list = []
rotten_rate_list = []
imdb_rate_list = []
star_rate_list = []
genre_list = []
runtime_list = []
director_list = []
synopsis_list = []
ac_list = []

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
import warnings
from selenium.webdriver.support.ui import WebDriverWait
warnings.filterwarnings("ignore")

def get_chrome_driver():
    # 1. 크롬 옵션 세팅
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("incognito")
    chrome_options.add_argument('disable-gpu')
    chrome_options.add_argument("headless")
    chrome_options.add_argument("no-sandbox")
    chrome_options.add_argument("disable-setuid-sandbox")
    chrome_options.add_argument("disable-dev-shm-usage")
    chrome_options.add_experimental_option("excludeSwitches", ['enable-logging'])

    # prefs = {'profile.default_content_setting_values': {'cookies' : 2, 'images': 2, 'plugins' : 2, 'popups': 2, 'geolocation': 2, 'notifications' : 2, 'auto_select_certificate': 2, 'fullscreen' : 2, 'mouselock' : 2, 'mixed_script': 2, 'media_stream' : 2, 'media_stream_mic' : 2, 'media_stream_camera': 2, 'protocol_handlers' : 2, 'ppapi_broker' : 2, 'automatic_downloads': 2, 'midi_sysex' : 2, 'push_messaging' : 2, 'ssl_cert_decisions': 2, 'metro_switch_to_desktop' : 2, 'protected_media_identifier': 2, 'app_banner': 2, 'site_engagement' : 2, 'durable_storage' : 2}}   
    # chrome_options.add_experimental_option('prefs', prefs)
    # 2. driver 생성하기
    driver = webdriver.Chrome(
        service=Service(ChromeDriverManager().install()), # 가장 많이 바뀐 부분
        options=chrome_options    
    )
    
    return driver

def get_driver(target_url):
    time.sleep(2)
    driver = get_chrome_driver()
    driver.get(target_url)
    wait=WebDriverWait(driver, 3)
    return driver

def extract_data(driver):
    #title_kr = driver.find_element_by_class_name('title-kr').text.strip()
    title_kr = driver.find_element_by_xpath('//*[@id="contents"]/div[2]/div[5]/div/h3').text.strip()
    title_en = driver.find_element_by_class_name('title-en').text.strip()
    genre = driver.find_element_by_xpath('//*[@id="contents"]/div[2]/div[5]/div[1]/p/span[1]').text.replace("·","")
    try:
        openingdate = driver.find_element_by_xpath('//*[@id="contents"]/div[2]/div[5]/div[1]/p/span[2]').text.replace("·","")
    except:
        openingdate = driver.find_element_by_xpath('//*[@id="contents"]/div[2]/div[5]/div[1]/p/span').text.replace("·","")
    
    rotten_rate = driver.find_element_by_class_name('rotten-wrap').text.strip()
    imdb_rate = driver.find_element_by_class_name('imdb-wrap').text.strip()
    star_rate = driver.find_element_by_class_name('movie-star-wrap').text.strip()
    synopsis = driver.find_element_by_class_name('synopsis').text.strip()
    director = driver.find_element_by_xpath('//*[@id="synopsis"]/article/div/div[1]/div[1]').text.replace("·","")
    runtime = driver.find_element_by_xpath('//*[@id="contents"]/div[4]/ul/li[1]').text.replace("#","")



    locates = "div.name"
    test_actor = driver.find_elements_by_css_selector(locates)
    test_actor = test_actor[1:]
    test_actor
    actors_list = []
    for test_actors in test_actor:
        test_actors = test_actors.text
        actors_list.append(test_actors)
    actors_list = ",".join(actors_list)


    title_list.append(title_kr)
    openingdate_list.append(openingdate)
    rotten_rate_list.append(rotten_rate)
    imdb_rate_list.append(imdb_rate)
    star_rate_list.append(star_rate)
    genre_list.append(genre)
    runtime_list.append(runtime)
    director_list.append(director)
    synopsis_list.append(synopsis)
    ac_list.append(actors_list)  

    # print('한글:',title_kr,","'영어:', title_en)
    print("장르:",genre, "개봉일:",openingdate,"상영시간:",runtime)    
    # print("평점",rotten_rate,"," ,imdb_rate,",", star_rate)
    # print("시놉시스:",synopsis)
    # print("감독:",director)
    # print("출연진:",actors_list)


for i in kino_netflix_all:
    target_url = "https://m.kinolights.com/title/{}".format(i)
    crawling = get_driver(target_url)
    extract_data(crawling)
    #crawling.quit()

In [62]:
crawling.quit()

In [7]:
df = pd.DataFrame({
    "제목":title_list,'개봉일':openingdate_list,'평점':rotten_rate_list, '장르':genre_list, '재생시간':runtime_list, '감독':director_list,"출연진":ac_list, "시놉시스":synopsis_list})
df.tail(5)

,제목,개봉일,평점,장르,재생시간,감독,출연진,시놉시스
57,모럴센스,2022,80% ·,멜로/로맨스,117분,박현진,"서주현,이준영,이엘,서현우,안승균,이석형,백현주",합의하에 색다른 파트너가 되기로 계약을 맺은 직장 동료. 두 사람의 아주 조금(?)...
58,밀애,2002,- % ·,드라마,110분,변영주,,격정 멜로 | 은밀한 탐닉 ...내가 잘했나요?\n서른의 전업주부 미흔의 집에 찾아...
59,고잉 인 스타일,2017,82% ·,코미디,96분,잭 브라프,"모건 프리먼,마이클 케인,알란 아킨,조이 킹,피터 세라피노윅,앤 마그렛,크리스토퍼 로이드",연금 동결이라니? 어느 날 갑자기 날벼락 같은 소식을 듣게 된 노년의 세 친구. 하...
60,드래곤 길들이기,2010,99% ·,코미디,98분,딘 데블로이스,"크리스 샌더스,제이 바루첼,제라드 버틀러,아메리카 페레라,크레이그 퍼거슨,조나 힐,...",용맹한 바이킹과 사나운 드래곤들의 싸움이 끊이지 않는 버크섬. 바이킹 족장의 아들 ...
61,아수라,2016,83% ·,액션,132분,김성수,"정우성,황정민,주지훈,곽도원,정만식,윤지혜,김해곤",강력계 형사 한도경은 이권과 성공을 위해 각종 범죄를 저지르는 악덕시장 박성배의 뒷...


In [68]:
df.to_excel("kino_netflix_632.xlsx")
df.to_csv("kino_netflix_632.csv")

In [24]:
driver = get_chrome_driver()



====== WebDriver manager ======
Current google-chrome version is 99.0.4844
Get LATEST chromedriver version for 99.0.4844 google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/99.0.4844.51/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\Kims\.wdm\drivers\chromedriver\win32\99.0.4844.51]


In [25]:
driver.quit()

In [176]:
#driver = get_chrome_driver()
time.sleep(2)
# 키노라이츠 홈페이지로 이동
target_url = "https://m.kinolights.com/title/70457/"
driver.get(target_url)

In [216]:
target_url = "https://m.kinolights.com/title/3879"
driver.get(target_url)

In [93]:
# review = driver.find_elements_by_class_name('review-content')[0].text
# print(review)
# type(review)

1. '해적: 도깨비 깃발'은 리뷰를 쓰기 복잡한 영화는 아니다. 오히려 아주 간단한 영화다. 그러나 나는 이 영화에 대한 생각을 정리하는데 고민이 깊어졌다. 이유는 전편 '해적: 바다로 간 산적'을 못봤기 때문이다. 결론부터 말하자면 '도깨비 깃발'은 썩 만족스런 영...


str

In [99]:
import warnings
warnings.filterwarnings("ignore")

In [203]:
driver.quit()

In [192]:
driver = get_chrome_driver()
target_url = "https://m.kinolights.com/title/41164/"
driver.get(target_url)



====== WebDriver manager ======
Current google-chrome version is 99.0.4844
Get LATEST chromedriver version for 99.0.4844 google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/99.0.4844.51/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\Kims\.wdm\drivers\chromedriver\win32\99.0.4844.51]


In [193]:
title_list =[]
openingdate_list = []
rotten_rate_list = []
imdb_rate_list = []
star_rate_list = []
genre_list = []
runtime_list = []
director_list = []
synopsis_list = []
ac_list = []

In [1]:
title_kr = driver.find_element_by_class_name('title-kr').text.strip()
title_en = driver.find_element_by_class_name('title-en').text.strip()
genre = driver.find_element_by_xpath('//*[@id="contents"]/div[2]/div[5]/div[1]/p/span[1]').text.replace("·","")
openingdate = driver.find_element_by_xpath('//*[@id="contents"]/div[2]/div[5]/div[1]/p/span[2]').text.replace("·","")
rotten_rate = driver.find_element_by_class_name('rotten-wrap').text.strip()
imdb_rate = driver.find_element_by_class_name('imdb-wrap').text.strip()
star_rate = driver.find_element_by_class_name('movie-star-wrap').text.strip()
synopsis = driver.find_element_by_class_name('synopsis').text.strip().replace("줄거리 :","").strip()
director = driver.find_element_by_xpath('//*[@id="synopsis"]/article/div/div[1]/div[1]').text.replace("·","")
runtime = driver.find_element_by_xpath('//*[@id="contents"]/div[4]/ul/li[1]').text.replace("#","")
locates = "div.name"
test_actor = driver.find_elements_by_css_selector(locates)
test_actor = test_actor[1:] 
test_actor
actors_list = []
for test_actors in test_actor:
    test_actors = test_actors.text
    actors_list.append(test_actors)
actors_list = ",".join(actors_list)


title_list.append(title_kr)
openingdate_list.append(openingdate)
rotten_rate_list.append(rotten_rate)
imdb_rate_list.append(imdb_rate)
star_rate_list.append(star_rate)
genre_list.append(genre)
runtime_list.append(runtime)
director_list.append(director)
synopsis_list.append(synopsis)
ac_list.append(actors_list)  

print('한글:',title_kr,","'영어:', title_en)
print(genre, openingdate, runtime)
print("평점",rotten_rate,"," ,imdb_rate,",", star_rate)
print(synopsis)
print(director)
print(actors_list)


NameError: name 'driver' is not defined

In [189]:
df = pd.DataFrame({
    "제목":title_list,'개봉일':openingdate_list,'평점':rotten_rate_list, '장르':genre_list, '재생시간':runtime_list, '감독':director_list,"출연진":ac_list, "시놉시스":synopsis_list})
df.tail(5)

,제목,개봉일,평점,장르,재생시간,감독,출연진,시놉시스
0,인셉션,2010,87% ·,드라마,147분,크리스토퍼 놀란,"와타나베 켄,레오나르도 디카프리오,마리옹 꼬띠아르,조셉 고든 레빗,엘리엇 페이지,톰...",줄거리 : 타인의 꿈에 들어가 생각을 훔치는 코브. 라이벌 기업의 정보를 빼내려는 ...


In [148]:
# locates = "div.name"
# test_actor = driver.find_elements_by_css_selector(locates)
# test_actor = test_actor[1:]
# test_actor
# ac_list = []
# for test_actors in test_actor:
#     test_actors = test_actors.text
#     ac_list.append(test_actors)
# ac_list = ",".join(ac_list)
# print(ac_list)

강하늘,한효주,이광수,권상우,채수빈,세훈,김성오


In [95]:
driver.quit()

In [174]:
driver.quit()